# 2. DBSCAN으로 학습하기 (iris)

In [92]:
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np

In [93]:
# data load

iris = load_iris()
wine = load_wine()

iris.data = pd.DataFrame(iris.data)
iris.target = pd.DataFrame(iris.target)

In [111]:
# Specify the eps value
from sklearn.cluster import DBSCAN
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors

'''
eps의 값은 항상 positive 해야 하므로 for문에 넣을 수 없다.
따라서 계획 했던대로 eps의 값을 증가 시켜보며 학습하는 것은 힘들고
minpts의 값을 증가시키면서 학습해보겠다.
'''


'\neps의 값은 항상 positive 해야 하므로 for문에 넣을 수 없다.\n따라서 계획 했던대로 eps의 값을 증가 시켜보며 학습하는 것은 힘들고\nminpts의 값을 증가시키면서 학습해보겠다.\n'

In [116]:
# class count graph with minpts change
# min_samples는 1~100까지 돌려본다.

class_count=[0,]

for i in range(1,100,1):
    dbscan = DBSCAN(min_samples=i)
    result = dbscan.fit_predict(iris.data)
    class_count.append(len(np.unique(result)))

class_count = pd.DataFrame(class_count)
class_count = class_count[class_count[0]==3]
print(class_count)

    0
5   3
6   3
7   3
8   3
9   3
10  3
14  3
15  3
16  3


In [118]:
# normal_iris_dbscan accuracy
# class의 개수가 데이터의 개수와 맞는 것의 accuracy를 측정해본다.

from sklearn.metrics.cluster import normalized_mutual_info_score

for i in class_count.index:
    dbscan = DBSCAN(min_samples=i)
    result = dbscan.fit_predict(iris.data)
    print('minpts=',i,":",normalized_mutual_info_score(iris.target[0], result))

minpts= 5 : 0.6044156375501387
minpts= 6 : 0.6010609730489903
minpts= 7 : 0.5825248082208494
minpts= 8 : 0.5842137354876208
minpts= 9 : 0.5871440773125185
minpts= 10 : 0.5912526142204563
minpts= 14 : 0.6338804521368303
minpts= 15 : 0.6145305039043192
minpts= 16 : 0.6083339269265271


In [150]:
# Scaler Preprocessed DBSCAN Accuracy

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer

data_minmax = MinMaxScaler().fit_transform(iris.data)
data_maxabs = MaxAbsScaler().fit_transform(iris.data)
data_standard = StandardScaler().fit_transform(iris.data)
data_robust = RobustScaler().fit_transform(iris.data)
data_normalize = Normalizer().fit_transform(iris.data)
data_log = np.log1p(iris.data)

class_count_minmax, class_count_maxabs, class_count_standard, class_count_robust, class_count_normalize, class_count_log = [],[],[],[],[],[]

for i in range(1,100,1):
    dbscan = DBSCAN(min_samples=i)
    result_minmax = dbscan.fit_predict(data_minmax)
    result_maxabs = dbscan.fit_predict(data_maxabs)
    result_standard = dbscan.fit_predict(data_standard)
    result_robust = dbscan.fit_predict(data_robust)
    result_normalize = dbscan.fit_predict(data_normalize)
    result_log = dbscan.fit_predict(data_log)

    class_count_minmax.append(len(np.unique(result_minmax)))
    class_count_maxabs.append(len(np.unique(result_maxabs)))
    class_count_standard.append(len(np.unique(result_standard)))
    class_count_robust.append(len(np.unique(result_robust)))
    class_count_normalize.append(len(np.unique(result_normalize)))
    class_count_log.append(len(np.unique(result_log)))

class_count_minmax = pd.DataFrame(class_count_minmax)
class_count_minmax = class_count_minmax[class_count_minmax[0]==3]
class_count_maxabs = pd.DataFrame(class_count_maxabs)
class_count_maxabs = class_count_maxabs[class_count_maxabs[0]==3]
class_count_standard = pd.DataFrame(class_count_standard)
class_count_standard = class_count_standard[class_count_standard[0]==3]
class_count_robust = pd.DataFrame(class_count_robust)
class_count_robust = class_count_robust[class_count_robust[0]==3]
class_count_normalize = pd.DataFrame(class_count_normalize)
class_count_normalize = class_count_normalize[class_count_normalize[0]==3]
class_count_log = pd.DataFrame(class_count_log)
class_count_log = class_count_log[class_count_log[0]==3]


In [151]:
score_minmax, score_maxabs, score_standard, score_robust, score_normalize, score_log = [], [], [], [], [], []

def minpts_score(arr, data, score_list):
    for i in arr.index:
        dbscan = DBSCAN(min_samples=i)
        result = dbscan.fit_predict(data)
        score_list.append(normalized_mutual_info_score(iris.target[0], result))

minpts_score(class_count_minmax, data_minmax, score_minmax)
minpts_score(class_count_maxabs, data_maxabs, score_maxabs)
minpts_score(class_count_standard, data_standard, score_standard)
minpts_score(class_count_robust, data_robust, score_robust)
minpts_score(class_count_normalize, data_normalize, score_normalize)
minpts_score(class_count_log, data_log, score_log)


In [149]:
def not_value(string, value):
    try:
        print(string, max(value))
    except ValueError:
        print(string, 0)

not_value('minmax score:', score_minmax)
not_value('maxabs score:', score_maxabs)
not_value('standard score:', score_standard)
not_value('robust score:', score_robust)
not_value('normalize score:', score_normalize)
not_value('log score:', score_log)

minmax score: 0
maxabs score: 0
standard score: 0.5168393648274324
robust score: 0.66438908517785
normalize score: 0
log score: 0


In [162]:
# apply PCA
from sklearn.decomposition import PCA

model = PCA(n_components=2)

data_minmax = MinMaxScaler().fit_transform(iris.data)
data_maxabs = MaxAbsScaler().fit_transform(iris.data)
data_standard = StandardScaler().fit_transform(iris.data)
data_robust = RobustScaler().fit_transform(iris.data)
data_normalize = Normalizer().fit_transform(iris.data)
data_log = np.log1p(iris.data)

data_minmax = model.fit_transform(data_minmax)
data_maxabs = model.fit_transform(data_maxabs)
data_standard = model.fit_transform(data_standard)
data_robust = model.fit_transform(data_robust)
data_normalize = model.fit_transform(data_normalize)
data_log = model.fit_transform(data_log)

class_count_minmax, class_count_maxabs, class_count_standard, class_count_robust, class_count_normalize, class_count_log = [],[],[],[],[],[]

for i in range(1,100,1):
    dbscan = DBSCAN(min_samples=i)
    result_minmax = dbscan.fit_predict(data_minmax)
    result_maxabs = dbscan.fit_predict(data_maxabs)
    result_standard = dbscan.fit_predict(data_standard)
    result_robust = dbscan.fit_predict(data_robust)
    result_normalize = dbscan.fit_predict(data_normalize)
    result_log = dbscan.fit_predict(data_log)

    class_count_minmax.append(len(np.unique(result_minmax)))
    class_count_maxabs.append(len(np.unique(result_maxabs)))
    class_count_standard.append(len(np.unique(result_standard)))
    class_count_robust.append(len(np.unique(result_robust)))
    class_count_normalize.append(len(np.unique(result_normalize)))
    class_count_log.append(len(np.unique(result_log)))

class_count_minmax = pd.DataFrame(class_count_minmax)
class_count_minmax = class_count_minmax[class_count_minmax[0]==3]
class_count_maxabs = pd.DataFrame(class_count_maxabs)
class_count_maxabs = class_count_maxabs[class_count_maxabs[0]==3]
class_count_standard = pd.DataFrame(class_count_standard)
class_count_standard = class_count_standard[class_count_standard[0]==3]
class_count_robust = pd.DataFrame(class_count_robust)
class_count_robust = class_count_robust[class_count_robust[0]==3]
class_count_normalize = pd.DataFrame(class_count_normalize)
class_count_normalize = class_count_normalize[class_count_normalize[0]==3]
class_count_log = pd.DataFrame(class_count_log)
class_count_log = class_count_log[class_count_log[0]==3]

In [163]:
score_minmax, score_maxabs, score_standard, score_robust, score_normalize, score_log = [], [], [], [], [], []

def minpts_score(arr, data, score_list):
    for i in arr.index:
        dbscan = DBSCAN(min_samples=i)
        result = dbscan.fit_predict(data)
        score_list.append(normalized_mutual_info_score(iris.target[0], result))

minpts_score(class_count_minmax, data_minmax, score_minmax)
minpts_score(class_count_maxabs, data_maxabs, score_maxabs)
minpts_score(class_count_standard, data_standard, score_standard)
minpts_score(class_count_robust, data_robust, score_robust)
minpts_score(class_count_normalize, data_normalize, score_normalize)
minpts_score(class_count_log, data_log, score_log)

In [164]:
def not_value(string, value):
    try:
        print(string, max(value))
    except ValueError:
        print(string, 0)

not_value('minmax score:', score_minmax)
not_value('maxabs score:', score_maxabs)
not_value('standard score:', score_standard)
not_value('robust score:', score_robust)
not_value('normalize score:', score_normalize)
not_value('log score:', score_log)

minmax score: 0
maxabs score: 0
standard score: 0.6496352668681216
robust score: 0.6716066484925268
normalize score: 0
log score: 0


In [165]:
# Apply TSNE
from sklearn.manifold import TSNE

model = TSNE(n_components=2, random_state=0)

data_minmax = MinMaxScaler().fit_transform(iris.data)
data_maxabs = MaxAbsScaler().fit_transform(iris.data)
data_standard = StandardScaler().fit_transform(iris.data)
data_robust = RobustScaler().fit_transform(iris.data)
data_normalize = Normalizer().fit_transform(iris.data)
data_log = np.log1p(iris.data)

data_minmax = model.fit_transform(data_minmax)
data_maxabs = model.fit_transform(data_maxabs)
data_standard = model.fit_transform(data_standard)
data_robust = model.fit_transform(data_robust)
data_normalize = model.fit_transform(data_normalize)
data_log = model.fit_transform(data_log)

class_count_minmax, class_count_maxabs, class_count_standard, class_count_robust, class_count_normalize, class_count_log = [],[],[],[],[],[]

for i in range(1,100,1):
    dbscan = DBSCAN(min_samples=i)
    result_minmax = dbscan.fit_predict(data_minmax)
    result_maxabs = dbscan.fit_predict(data_maxabs)
    result_standard = dbscan.fit_predict(data_standard)
    result_robust = dbscan.fit_predict(data_robust)
    result_normalize = dbscan.fit_predict(data_normalize)
    result_log = dbscan.fit_predict(data_log)

    class_count_minmax.append(len(np.unique(result_minmax)))
    class_count_maxabs.append(len(np.unique(result_maxabs)))
    class_count_standard.append(len(np.unique(result_standard)))
    class_count_robust.append(len(np.unique(result_robust)))
    class_count_normalize.append(len(np.unique(result_normalize)))
    class_count_log.append(len(np.unique(result_log)))

class_count_minmax = pd.DataFrame(class_count_minmax)
class_count_minmax = class_count_minmax[class_count_minmax[0]==3]
class_count_maxabs = pd.DataFrame(class_count_maxabs)
class_count_maxabs = class_count_maxabs[class_count_maxabs[0]==3]
class_count_standard = pd.DataFrame(class_count_standard)
class_count_standard = class_count_standard[class_count_standard[0]==3]
class_count_robust = pd.DataFrame(class_count_robust)
class_count_robust = class_count_robust[class_count_robust[0]==3]
class_count_normalize = pd.DataFrame(class_count_normalize)
class_count_normalize = class_count_normalize[class_count_normalize[0]==3]
class_count_log = pd.DataFrame(class_count_log)
class_count_log = class_count_log[class_count_log[0]==3]

In [166]:
score_minmax, score_maxabs, score_standard, score_robust, score_normalize, score_log = [], [], [], [], [], []

def minpts_score(arr, data, score_list):
    for i in arr.index:
        dbscan = DBSCAN(min_samples=i)
        result = dbscan.fit_predict(data)
        score_list.append(normalized_mutual_info_score(iris.target[0], result))

minpts_score(class_count_minmax, data_minmax, score_minmax)
minpts_score(class_count_maxabs, data_maxabs, score_maxabs)
minpts_score(class_count_standard, data_standard, score_standard)
minpts_score(class_count_robust, data_robust, score_robust)
minpts_score(class_count_normalize, data_normalize, score_normalize)
minpts_score(class_count_log, data_log, score_log)

In [169]:
def not_value(string, value):
    try:
        print(string, max(value))
    except ValueError:
        print(string, 0)

not_value('minmax score:', score_minmax)
not_value('maxabs score:', score_maxabs)
not_value('standard score:', score_standard)
not_value('robust score:', score_robust)
not_value('normalize score:', score_normalize)
not_value('log score:', score_log)

minmax score: 0.27251776618860857
maxabs score: 0
standard score: 0.2124641071366041
robust score: 0
normalize score: 0.28079853190582854
log score: 0
